# Clustering Methods

> Weitong Zhang
> 2015011493
>
> <zwt15@mails.tsinghua.edu.cn>

## No empty subset in k-means

First of all, let's suppose that there are at least one empty subset $D_\phi$ generated by k-means clustering method, since in most of the case, $c \le n$ (otherwise, the k-means method is useless), therefore, the number of non-empty subset $c_{D\ne\phi}$ would be less than $n$

According to the pigeonhole principle, there are at least one subset which contains at least 2 samples, now lets set this subset as $D_\kappa$

Therefore, we can reconstruct the subset by moving the first sample $D_{\kappa0}$ to one empty subset, and keep the other subset fixed, therefore we remove an empty subset and get a subset with only one sample.

Let's assume that the original division strategy would lead to a loss determined by $J$, while the new one will have a loss defined by $J'$, we can easily conclude that:

$$J - J' = \sum_{i=0, D_{\kappa}}(x_i - m_{D_\kappa})^2 - \sum_{i=1, D_{\kappa'}}(x_i - m_{D_\kappa'})^2 \ge  \sum_{i=1, D_{\kappa}}(x_i - m_{D_\kappa})^2
- \sum_{i=1, D_{\kappa'}}(x_i - m_{D_\kappa'})^2$$

Note that it is obvious that the subset with only one sample have no loss

In the subset of $D{\kappa}'$, it is obvious that the center $m_{D_\kappa'}$ will lead to the lowest loss in function $\sum_{i=1, D_{\kappa'}}(x_i - m)^2$, that is to say, $\sum_{i=1, D_{\kappa}}(x_i - m_{D_\kappa})^2
- \sum_{i=1, D_{\kappa'}}(x_i - m_{D_\kappa'})^2 \ge 0$

Therefore, $J - J' \ge 0$, $J = J'$ only when in each subset, all of the samples are the same point, which makes any clustering method useless.

In conclusion, once we can find a division strategy with a empty subset, we can find one with this subset non-empty and improve the performance of the clustering, that is to say, no empty subset will be provided by k-means method.

## Programming

### Load the MNIST dataset

In [1]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

ModuleNotFoundError: No module named 'scipy.sparse._sparsetools'